In [1]:
print("ok")

ok


In [4]:
import os

In [5]:
from __future__ import annotations

import os
from time import sleep

import leetcode

# Initialize client
configuration = leetcode.Configuration()

# NOTE: cookies var is just a dict with `csrftoken` and `LEETCODE_SESSION`
# fields which contain corresponding cookies from web browser
leetcode_session: str = os.environ["LEETCODE_SESSION"]

csrf_token = os.environ["CSRF_TOKEN"]

configuration.api_key["x-csrftoken"] = csrf_token
configuration.api_key["csrftoken"] = csrf_token
configuration.api_key["LEETCODE_SESSION"] = leetcode_session
configuration.api_key["Referer"] = "https://leetcode.com"
configuration.debug = False

api_instance = leetcode.DefaultApi(leetcode.ApiClient(configuration))


In [6]:
code = """
class Solution {
public:
    void bfs(int prevcolor,int row,int col,int color,vector<vector<int>>&visited,vector<vector<int>>&image)
    {
        // it'll ease the work
        queue<pair<int,int>> q;
        q.push({row,col});
        visited[row][col]=1;

        while(!q.empty())
        {
            int r = q.front().first;
            int c = q.front().second;
            q.pop();

            if(image[r][c]==prevcolor) 
              image[r][c]=color;
            
            // up
            if(r-1>=0 && !visited[r-1][c] && image[r-1][c]==prevcolor) 
              {q.push({r-1,c}); visited[r-1][c]=1;image[r-1][c]=color;}
            // right
            if(c+1<image[0].size() &&  !visited[r][c+1] && image[r][c+1]==prevcolor) 
              {q.push({r,c+1}); visited[r][c+1]=1;image[r][c+1]=color;}
            //down
            if(r+1<image.size() && !visited[r+1][c] && image[r+1][c]==prevcolor) 
              {q.push({r+1,c}); visited[r+1][c]=1; image[r+1][c]=color;}

            // left
            if(c-1>=0 && !visited[r][c-1] && image[r][c-1]==prevcolor) 
              {q.push({r,c-1}); visited[r][c-1]=1;image[r][c-1]=color;}
        }
    }
    vector<vector<int>> floodFill(vector<vector<int>>& image, int sr, int sc, int color) {
          int prevcolor = image[sr][sc];

          // visiting array.
          vector<vector<int>> visited(image.size(),vector<int>(image[0].size(),0));

          bfs(prevcolor,sr,sc,color,visited,image);

          return image;
    }
};
"""

In [10]:
submission = leetcode.Submission(
    judge_type="large", typed_code=code, question_id=733, test_mode=False, lang="cpp"
)

submission_id = api_instance.problems_problem_submit_post(
    problem="flood-fill", body=submission
)


In [11]:
print("Submission has been queued. Result:")
print(submission_id)

sleep(5)  # FIXME: should probably be a busy-waiting loop

submission_result = api_instance.submissions_detail_id_check_get(
    id=submission_id.submission_id
)

Submission has been queued. Result:
{'submission_id': 1584728626}


In [12]:
print("Got submission result:")
print(submission_result)


Got submission result:
{'status_code': 10, 'lang': 'cpp', 'run_success': True, 'status_runtime': '0 ms', 'memory': 18668000, 'display_runtime': '0', 'question_id': 733, 'elapsed_time': 65, 'compare_result': '11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111', 'code_output': '', 'std_output': '', 'last_testcase': '', 'expected_output': '', 'task_finish_time': 1742839404544, 'task_name': 'judger.judgetask.Judge', 'finished': True, 'total_correct': 278, 'total_testcases': 278, 'runtime_percentile': 100, 'status_memory': '18.7 MB', 'memory_percentile': 10.55539999999999, 'pretty_lang': 'C++', 'submission_id': '1584728626', 'status_msg': 'Accepted', 'state': 'SUCCESS'}
